In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Receptive Field Arithmetic

```
JumpOut = JumpIn * stride
RFout = RFin + (KernelSize - 1) * JumpIn
```

In [7]:
# Original Model
class Net0(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) # 28x28x1 -> 28x28x32 [Jin=1,K=3,RFin=1] RF: 3, Jout: 1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # -> 28x28x64 [Jin=1,K=3,RFin=3] RF: 5, Jout: 1
        self.pool1 = nn.MaxPool2d(2, 2) # -> 14x14x64 [Jin=1,K=2,RFin=5] RF: 6, Jout: 2
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # -> 14x14x128 [Jin=2,K=3,RFin=6] RF: 10, Jout: 2
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # -> 14x14x256 [Jin=2,K=3,RFin=10] RF: 14, Jout: 2
        self.pool2 = nn.MaxPool2d(2, 2) # -> 7x7x256 [Jin=2,K=2,RFin=14] RF: 16, Jout: 4
        self.conv5 = nn.Conv2d(256, 512, 3) # -> 5x5x512 [Jin=4,K=3,RFin=16] RF: 24, Jout: 4
        self.conv6 = nn.Conv2d(512, 1024, 3) # -> 3x3x1024 [Jin=4,K=3,RFin=24] RF: 32, Jout: 4
        self.conv7 = nn.Conv2d(1024, 10, 3) # -> 1x1x10 [Jin=4,K=3,RFin=32] RF: 40, Jout: 4

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

# Number of Parameters = 6.3M
# Batch Size = 128
# 1st epoch Acc = 29%

#-------------------------------------------------------------------------------

# Improved Model - 1
class Net1(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) # 28x28x1 -> 28x28x32 [Jin=1,K=3,RFin=1] RF: 3, Jout: 1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # -> 28x28x64 [Jin=1,K=3,RFin=3] RF: 5, Jout: 1
        self.pool1 = nn.MaxPool2d(2, 2) # -> 14x14x64 [Jin=1,K=2,RFin=5] RF: 6, Jout: 2
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # -> 14x14x128 [Jin=2,K=3,RFin=6] RF: 10, Jout: 2
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # -> 14x14x256 [Jin=2,K=3,RFin=10] RF: 14, Jout: 2
        self.pool2 = nn.MaxPool2d(2, 2) # -> 7x7x256 [Jin=2,K=2,RFin=14] RF: 16, Jout: 4
        self.conv5 = nn.Conv2d(256, 512, 3) # -> 5x5x512 [Jin=4,K=3,RFin=16] RF: 24, Jout: 4
        self.conv6 = nn.Conv2d(512, 1024, 3) # -> 3x3x1024 [Jin=4,K=3,RFin=24] RF: 32, Jout: 4
        self.conv7 = nn.Conv2d(1024, 10, 3) # -> 1x1x10 [Jin=4,K=3,RFin=32] RF: 40, Jout: 4

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

# Improvements from original model
# * Remove RELU from last Conv layer

# Number of Parameters = 6.3M
# Batch Size = 128
# 1st epoch Acc = 98%

#-------------------------------------------------------------------------------

# Improved Model - 2
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        input = lambda in_channels, out_channels: nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
        conv = lambda in_channels, out_channels: nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
        transition = lambda in_channels, out_channels: nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        output = lambda in_channels, out_channels: nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3),
        )

        self.input = input(1, 16)
        self.conv1 = conv(16, 32)
        self.transition1 = transition(32, 16)
        self.conv2 = conv(16, 32)
        self.transition2 = transition(32, 16)
        self.output = output(16, 10)

    def forward(self, x):
        x = self.input(x)
        x = self.conv1(x)
        x = self.transition1(x)
        x = self.conv2(x)
        x = self.transition2(x)
        x = self.output(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)
        
# Improvements from original model
# * Remove RELU from last Conv layer
# * Change Batch Size = 64 -> No change, so reverted back to 128
# * Reduced number of parameters to less than 20K
# * Refactored network to be use nn.Sequential
# * Add transition block, this becomes `excite and squeeze` network
# * Added Batch Norm after every conv layer, except last layer

# Number of Parameters = 16.8K
# Batch Size = 128
# Max accuracy reached @18th epoch:
#   loss=0.0036736882757395506 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.12it/s]
#   0%|          | 0/469 [00:00<?, ?it/s]
#   Test set: Average loss: 0.0218, Accuracy: 9936/10000 (99%)

# Training loss is 0.0036, but test loss is 0.0218... Looks to be overfitting
# Two ways to fix overfitting (there are many, but following are two from them):
# 1) Add Image Preprocessing:
#    ```
#    transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
#    transforms.ColorJitter(brightness=0.2, contrast=0.2),
#    ```
#   Reached 99.45 @10th epoch:
#    loss=0.010153046809136868 batch_id=468: 100%|██████████| 469/469 [00:56<00:00,  8.31it/s]
#    0%|          | 0/469 [00:00<?, ?it/s]
#    Test set: Average loss: 0.0180, Accuracy: 9945/10000 (99%)

#   Reached 99.50 @20th epoch:
#    loss=0.03761047497391701 batch_id=468: 100%|██████████| 469/469 [00:56<00:00,  8.36it/s]
#    Test set: Average loss: 0.0164, Accuracy: 9950/10000 (100%)

# 2) Dropout: No luck, tried 0.1,0.2, didn't get expected results

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
              ReLU-3           [-1, 16, 28, 28]               0
            Conv2d-4           [-1, 32, 28, 28]           4,640
       BatchNorm2d-5           [-1, 32, 28, 28]              64
              ReLU-6           [-1, 32, 28, 28]               0
            Conv2d-7           [-1, 16, 28, 28]             528
       BatchNorm2d-8           [-1, 16, 28, 28]              32
              ReLU-9           [-1, 16, 28, 28]               0
        MaxPool2d-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 32, 14, 14]           4,640
      BatchNorm2d-12           [-1, 32, 14, 14]              64
             ReLU-13           [-1, 32, 14, 14]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        transforms.ColorJitter(brightness=0.2, contrast=0.2),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0550, Accuracy: 9828/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0766, Accuracy: 9772/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0182, Accuracy: 9934/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0181, Accuracy: 9940/10000 (99%)



loss=0.03761047497391701 batch_id=468: 100%|██████████| 469/469 [00:56<00:00,  8.36it/s]



Test set: Average loss: 0.0164, Accuracy: 9950/10000 (100%)

